In [6]:
# ================================
# Cell 1 — Functions
# ================================
import os, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from typing import Optional, Tuple, List

from qiskit import QuantumCircuit
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.visualization import plot_histogram

# AerSimulator（環境差異に備えて両対応）
try:
    from qiskit_aer import AerSimulator
except Exception:
    # 古い環境向けフォールバック
    from qiskit import Aer
    AerSimulator = lambda: Aer.get_backend("aer_simulator")


# ---------- 小道具 ----------
def rmse(y_true, y_pred):
    y_true = np.asarray(y_true, float).ravel()
    y_pred = np.asarray(y_pred, float).ravel()
    return float(np.sqrt(np.mean((y_true - y_pred) ** 2)))

def minmax_pm1(x: np.ndarray) -> Tuple[np.ndarray, float, float]:
    x = np.asarray(x, float)
    mn, mx = float(np.min(x)), float(np.max(x))
    if mx == mn: return np.zeros_like(x), mn, mx
    return 2.0*(x - mn)/(mx - mn) - 1.0, mn, mx

def load_series(csv_path: str = "./Dataset_Simulated_Price_swaption.csv",
                take_col: Optional[str] = None,
                max_len: int = 64) -> Tuple[np.ndarray, Optional[np.ndarray], str]:
    """CSVがあればDateと最初の数値列（またはtake_col）を使う。なければ疑似データ。"""
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path, engine="python", sep=None)
        # Date列推定
        date_col = next((c for c in df.columns[::-1] if str(c).strip().lower()=="date"), df.columns[-1])
        # 数値候補
        cand = [c for c in df.columns if c != date_col]
        use_col = take_col if (take_col in cand) else cand[0]
        s = pd.to_numeric(df[use_col], errors="coerce").astype(float).values
        if len(s) > max_len:
            dates = pd.to_datetime(df[date_col].values[:max_len], errors="coerce")
            s = s[:max_len]
        else:
            dates = pd.to_datetime(df[date_col].values, errors="coerce")
        return s, dates, str(use_col)
    # 疑似データ
    T = max_len
    t = np.arange(T)
    s = 0.03 + 0.01*np.sin(2*np.pi*t/24) + 0.003*np.sin(2*np.pi*t/7) + 0.001*np.random.randn(T)
    return s.astype(float), None, "Synthetic"


# ---------- QRC（最小構成：RZ→CX列→RX） ----------
def _mix_layer(qc: QuantumCircuit, n_qubits: int):
    # 元ノートのミックス層（RZ→CX列→(末端)CX→RX）を2量子ビット用に簡略化。:contentReference[oaicite:1]{index=1}
    for q in range(n_qubits): qc.rz((q+1)*0.07, q)
    if n_qubits >= 2:
        for q in range(n_qubits-1): qc.cx(q, q+1)
    for q in range(n_qubits): qc.rx((q+1)*0.13, q)

def build_qrc_window_circuits(window_pm1: np.ndarray, n_qubits=2, depth=1, kappa_in=1.0) -> List[QuantumCircuit]:
    """
    窓Wの各ステップsの「注入(θ=κ*x_s)→ミックス×depth→測定」を1本の回路に。
    s=0..W-1 で W本返す（Samplerに素直）。
    """
    W = int(len(window_pm1)); circs = []
    for s in range(W):
        qc = QuantumCircuit(n_qubits, name=f"qrc_s{s}")
        for t in range(s+1):
            ang = float(kappa_in * window_pm1[t])
            for q in range(n_qubits):
                qc.ry(ang, q)
            for _ in range(depth): _mix_layer(qc, n_qubits)
        qc.measure_all()
        circs.append(qc)
    return circs

def _expZ_from_quasi(quasi, n_qubits: int) -> np.ndarray:
    """quasi({bit->p})→<Z_i>."""
    eZ = np.zeros(n_qubits, float)
    for bit, p in quasi.items():
        if isinstance(bit, int):
            bs = format(bit, f"0{n_qubits}b")[::-1]  # LSBをq0に合わせて反転
        else:
            bs = bit[::-1]
        for i, ch in enumerate(bs[:n_qubits]):
            eZ[i] += p*(+1.0 if ch=='0' else -1.0)
    return eZ

def quasi_to_counts(quasi, n_qubits: int, shots: int) -> dict:
    """可視化用：quasiを近似カウントに変換（合計≈shots）。"""
    counts = {}
    total = 0
    for bit, p in quasi.items():
        key = format(bit, f"0{n_qubits}b") if isinstance(bit, int) else bit
        c = int(max(0, round(p * shots)))
        counts[key] = counts.get(key, 0) + c
        total += c
    # 正規化調整
    if total != shots and total > 0:
        scale = shots / total
        for k in counts: counts[k] = int(round(counts[k]*scale))
    return counts


# ---------- Sampler（ローカル / 実機） ----------
def run_local_sampler(circuits: List[QuantumCircuit], shots: int = 1024):
    """AerSimulatorへトランスパイル → ローカルSamplerで実行。"""
    backend = AerSimulator()
    pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
    isa_circs = pm.run(circuits)

    # Sampler（ローカル）は環境により場所が異なることがあるので防御的に
    try:
        from qiskit.primitives import Sampler as LocalSampler
        sampler = LocalSampler()
        job = sampler.run(isa_circs, shots=shots)
    except Exception:
        from qiskit_aer.primitives import Sampler as AerSampler
        sampler = AerSampler(backend=backend)
        try:
            job = sampler.run(isa_circs, shots=shots)
        except TypeError:
            job = sampler.run(isa_circs)

    res = job.result()
    quasi_list = getattr(res, "quasi_dists", getattr(res, "quasi_distributions", None))
    if quasi_list is None:
        raise RuntimeError("Sampler結果からquasi分布を取得できませんでした。")
    return quasi_list, isa_circs, backend


def _choose_backend_name(service, min_qubits=2, prefer: Optional[str]="ibm_torino") -> str:
    """最小pending_jobsの実機。使えなければpreferを返す。"""
    try:
        cands = []
        for b in service.backends():
            try:
                name = b.name
                sim = getattr(b, "simulator", None)
                if sim is None and hasattr(b, "configuration"):
                    cfg = b.configuration()
                    sim = getattr(cfg, "simulator", False)
                    n_qubits = getattr(cfg, "n_qubits", None)
                else:
                    n_qubits = getattr(b, "num_qubits", None)
                if sim: continue
                if n_qubits is not None and n_qubits < min_qubits: continue
                st = b.status()
                if not getattr(st, "operational", True): continue
                pend = getattr(st, "pending_jobs", 10**9)
                cands.append((pend, name))
            except Exception:
                continue
        if cands:
            cands.sort(key=lambda x: (x[0], x[1]))
            return cands[0][1]
    except Exception:
        pass
    return prefer


def try_run_hw_sampler(circuits: List[QuantumCircuit], shots: int, min_qubits=2,
                       preferred: str = "ibm_torino"):
    """
    実機到達を試み、失敗（DNS等）なら None で返す。
    戻り値: dict | None
    """
    try:
        from qiskit_ibm_runtime import QiskitRuntimeService, Sampler as RTSampler, Session, Options
    except Exception as e:
        warnings.warn(f"qiskit-ibm-runtime が見つかりません: {e}")
        return None

    # 認証（保存済み or 環境変数 / Cell2のsave_account後を想定）
    try:
        service = QiskitRuntimeService()
    except Exception as e:
        warnings.warn(f"Runtime認証に失敗: {e}")
        return None

    # 実機選択→PassManager→Sampler実行
    try:
        backend_name = _choose_backend_name(service, min_qubits=min_qubits, prefer=preferred)
        backend_obj = service.backend(backend_name)
        print(f"[HW] selected backend = {backend_name} (pending_jobs={getattr(backend_obj.status(), 'pending_jobs', 'NA')})")

        pm = generate_preset_pass_manager(backend=backend_obj, optimization_level=3)
        isa_hw = pm.run(circuits)

        # Runtime Sampler
        try:
            opts = Options()
            try: opts.default_shots = shots
            except Exception: pass
            with Session(service=service, backend=backend_name) as sess:
                sampler = RTSampler(session=sess, options=opts)
                job = sampler.run(isa_hw, shots=shots)
                print("job id:", job.job_id())
                # ここでresult()を呼ぶと待ちになる可能性 → それでも取得を試みる（失敗は捕捉）
                res = None
                try:
                    res = job.result()
                except Exception as e:
                    print(f"[HW] result() 取得に失敗（後で再実行可）: {e}")
                return {"backend": backend_name, "job": job, "result": res, "isa_hw": isa_hw}
    except Exception as e:
        warnings.warn(f"実機実行に失敗（フォールバックします）: {e}")
        return None


SyntaxError: expected 'except' or 'finally' block (2370152861.py, line 215)

In [5]:
# ================================
# Cell 2 — Train (use token + pick least busy real backend)
# ================================
from qiskit_ibm_runtime import QiskitRuntimeService

# --- 0) 認証：提供いただいたトークンを使用（保存せずインメモリ利用） ---
TOKEN = "P4MVm0GVTVkrniQ3nkSfGAhhuZaPHD3R32uyEwpzuLIO"  # ←実行後は削除/再発行推奨
try:
    # トークン直渡しでセッション利用（ローカルに保存しない）
    service = QiskitRuntimeService(channel="ibm_quantum", token=TOKEN)
except Exception:
    # 失敗時はアカウント保存→読込（上書き許可）
    QiskitRuntimeService.save_account(channel="ibm_quantum", token=TOKEN, overwrite=True)
    service = QiskitRuntimeService()

# --- 1) バックエンド選択（優先: ibm_torino → 混雑最小） ---
PREFERRED = "ibm_torino"
cfg = QRCConfig(n_qubits=2, depth=1, W=3, kappa=1.0)  # 超軽量設定（実機向け）
BACKEND_NAME = pick_least_busy_backend(service, min_qubits=cfg.n_qubits, preferred=PREFERRED)

# 実行規模（混雑に応じて小さめに）
SHOTS = 2000  # 1000〜2000程度が現実的

print(f"[Backend] selected = {BACKEND_NAME} (preferred={PREFERRED})")
print(f"[Job size] W={cfg.W}, n_qubits={cfg.n_qubits}, depth={cfg.depth}, shots={SHOTS}")

# --- 2) データ（CSVがあれば使う／無ければ合成） ---
CSV_PATH = "./Dataset_Simulated_Price_swaption.csv"  # 任意。無ければ合成に自動フォールバック
SERIES = prepare_series(CSV_PATH, column=None)

# --- 3) ウィンドウ化（H=1のレベル予測） ---
Xw, y = make_windows_1d(SERIES, W=cfg.W, H=1)

# 実ジョブを小さく（パラメタセット本数を抑制）
N_MAX = 48
Xw, y = Xw[:N_MAX], y[:N_MAX]

# --- 4) 入力スケーリング → 角度に変換 ---
scaler_in = StandardScaler().fit(Xw[:int(0.7 * len(Xw))])
thetas = windows_to_thetas(Xw, scaler_in, cfg)

# --- 5) 実機で <Z> 特徴量を取得（Sampler + Session） ---
X_feat = qrc_features_sampler(service, BACKEND_NAME, cfg, thetas,
                              shots=SHOTS, use_pair=False)  # TrueでZZ相関も可

# --- 6) 学習/評価（Ridge） ---
n_tr = int(0.7 * len(X_feat))
X_tr, X_te = X_feat[:n_tr], X_feat[n_tr:]
y_tr, y_te = y[:n_tr], y[n_tr:]

mdl = Ridge(alpha=0.1, fit_intercept=True).fit(X_tr, y_tr)
y_pred = mdl.predict(X_te)

# --- 7) Cell 3 用の共有オブジェクト ---
TRAINING_SUMMARY = {
    "backend": BACKEND_NAME,
    "shots": SHOTS,
    "n_qubits": cfg.n_qubits,
    "W": cfg.W,
    "N_total": int(len(X_feat)),
    "N_train": int(len(X_tr)),
    "RMSE": rmse(y_te, y_pred),
    "MAE": float(mean_absolute_error(y_te, y_pred)),
}
Y_TEST, Y_PRED = y_te, y_pred


RequestsApiError: 'HTTPSConnectionPool(host=\'auth.quantum.ibm.com\', port=443): Max retries exceeded with url: /api/version (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x11f8f1520>: Failed to resolve \'auth.quantum.ibm.com\' ([Errno 8] nodename nor servname provided, or not known)"))'

In [ ]:
# ================================
# Cell 3 — Show Results
# ================================
print("=== Minimal QRC on real hardware — Summary ===")
for k, v in TRAINING_SUMMARY.items():
    print(f"{k}: {v}")

parity_plot(Y_TEST, Y_PRED, title="Parity (test)")
